<a href="https://colab.research.google.com/github/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/blob/main/Sentiment_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd

##Szószedet

In [33]:
!rm *zip*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip
df=pd.read_csv("TEST.csv")

--2021-10-06 17:36:29--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-06 17:36:29--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [34]:
df.head()
alltext1=df.text
print (len(alltext1))

2640


In [35]:
df2=pd.read_csv("TRAIN.csv")
alltext2=df2.text
print (len(alltext2))

12000


In [31]:
alltext1=list(alltext1)
alltext2=list(alltext2)
alltext=alltext1+alltext2
print(f" {len(alltext)}")


 14640


In [36]:
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()
print(spell.correction("@frustrating"))

frustrating


In [37]:
import re

def remove_URL(text):
    """Remove URLs from a text string"""
    return re.sub(r"http\S+", "", text)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def is_number(inp:str):
    a=re.search("[\d]",inp)
    if a==None: 
        return(False)
    return(True)

def strip_all_entities(text):
    import re,string
    entity_prefixes = ['“','@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def query_sentence(index):
    print(f"orig: {alltext[index]}")
    spell=SpellChecker()
    inp=alltext[index]
    inp=remove_URL(inp)
    inp=remove_emoji(inp)
    inp=strip_all_entities(inp)
    
    txt=inp.replace(","," ").replace("."," ").replace("!"," ").replace("#"," ").replace("???"," ").replace("??"," ").replace("?"," ").replace("  "," ").replace("  "," ").replace('"',"").replace("“","")

    #print(txt)
    normal=""
    for i in txt.split(" "):
        w1=i.lower().strip()
        if w1 !="":
            w2=spell.correction(w1).strip()
        else:
            w2=""
        if w2=="i":
            pass
            #print(f"chek: {w1}, {w2}")
        if is_number(w2):
            w2=""    
        normal += w2+" "

    print(f"  conv: {normal}")
    return(normal)

##Mentés

In [ ]:
t=[]
for i in range(len(alltext)):
    print(f"{i}", end=" ")
    t.append(query_sentence(i))


In [49]:
s=set()
for sent in t:
    wlist=sent.strip().split()
    for w in wlist:
        s.add(w)
print (len(s))
ordered=list(s)
ordered.sort()

9098


In [ ]:
print(ordered)

In [63]:
sentence=pd.DataFrame(t)
sentence.columns=["Sentence"]
sentence.to_csv("Sentences.csv")

In [55]:
wordlist=pd.DataFrame(ordered)
wordlist.columns=["Words"]
wordlist.to_csv("words.csv")

###---------------------------------


###Adatbetöltés SZAVAK BETÖLTÉSE

In [64]:
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv"

In [65]:
!wget $words_url

--2021-10-06 19:00:18--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv [following]
--2021-10-06 19:00:18--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114745 (112K) [text/plain]
Saving to: ‘words.csv’

words.csv           100%[=================

In [66]:
words=pd.read_csv("words.csv",index_col=0)
words.head()
words_list=list(words["Words"])

In [67]:
word_dict={v:i+1 for i,v in enumerate(words_list)}

### adatbetöltés mondatok betöltése

In [80]:
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/Sentences.csv"

In [83]:
sentences.head()

,Sentence
0,well someone should tell that to the employees...
1,let i all have a extraordinary week and make i...
2,since i have an international connection at am...
3,woulda been nice of you to let us know that or...
4,cond they put her on am elite tonite i think ...


In [ ]:
!wget $sentences_url
sentences=pd.read_csv("Sentences.csv",index_col=0)
sentences.head()
sentences_list=list(sentences["Sentence"])

In [ ]:
sentences_list[2640:]

##Tanulás

In [89]:
df=pd.read_csv("TRAIN.csv")

In [90]:
df["clean_text"]=sentences_list[2640:]

In [94]:
all_sent=[]
for sentence in df.clean_text:
    sent_list=sentence.strip().split()
    out=[]
    for word in sent_list:
        out.append(word_dict[word]) 
    all_sent.append(out)

In [95]:
df["sentence_code"]=all_sent

In [96]:
table=list(df["sentence_code"]) # typo
lens=max([len(x) for x in table])
sentiment=list(df["airline_sentiment"])


In [97]:
ytrain=[]
for sent in sentiment:
    if sent=="negative":
        o=[1,0,0]
    if sent=="neutral":
        o=[0,1,0]
    if sent=="positive":
        o=[0,0,1]
    ytrain.append(o)
    

In [98]:
x0=[0 for _ in range(lens) ]
xtrain=[]
for sent in table:
    o1=list(x0[0:lens-len(sent)]+list(sent))
    xtrain.append(o1)

In [99]:
print(lens)

33


In [100]:


lstm_size=lens
max_input_length=lens
embedding_size=100
n_words=len(table)

In [101]:

# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy
from tensorflow.keras import backend as K
# Gondolj bele, hogy a hálóban az első réteg egy "beágazás" kell majd legyen
# Ne feledd behozni a funkcionális vagy szekvenciális API-nak megfelelő "fő" osztályt
# Adott esetben az optimalizálót
# Nem külömben a "bakcendet", hogy jó gyakorlat szerint reseteld a gráfot
#tf.reset_default_graph()
K.clear_session
x= Input(shape=(max_input_length,))
embedded_x=Embedding(n_words+1,embedding_size, input_length=max_input_length-1, mask_zero=True)(x)
lstm_output=LSTM(lstm_size,return_sequences=True)(embedded_x)
lstm_output=LSTM(lstm_size,return_sequences=False)(lstm_output)
predictions= Dense(3, activation="softmax")(lstm_output)
model=Model(inputs=x, outputs=predictions)



In [102]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 33, 100)           1200100   
_________________________________________________________________
lstm (LSTM)                  (None, 33, 33)            17688     
_________________________________________________________________
lstm_1 (LSTM)                (None, 33)                8844      
_________________________________________________________________
dense (Dense)                (None, 3)                 102       
Total params: 1,226,734
Trainable params: 1,226,734
Non-trainable params: 0
_________________________________________________________________


In [103]:
x

<KerasTensor: shape=(None, 33) dtype=float32 (created by layer 'input_1')>

In [104]:
import numpy as np

In [105]:
x_train=xtrain[0:-2000]
y_train=ytrain[0:-2000]
x_test=xtrain[-2000:]
y_test=ytrain[-2000:]

In [106]:
train_x = np.asarray(xtrain)
train_y = np.asarray(ytrain)

In [107]:
# Loss 

loss = categorical_crossentropy # One-hot enkódolt kimenetünk van. Mit is használunk?

# Optimizer
optimizer = Adam() #Ízlés szerint...
 
# Compilation
#############

model.compile(optimizer=optimizer,loss=loss)

In [108]:
# Illesszük az adatra a modellt. Használjunk 10% validációt
history=model.fit(x=xtrain,y=ytrain,validation_data=( x_test,y_test), epochs=25, batch_size=100)
# - nyelvmodellnél ez nem olyan lényeges
# Használhatjuk a Keras beépített validációs splitjét.
# Adjunk meg reális batch méretet!

Epoch 1/25
120/120 [==============================] - 16s 89ms/step - loss: 0.7473 - val_loss: 0.5179
Epoch 2/25
120/120 [==============================] - 9s 72ms/step - loss: 0.4758 - val_loss: 0.3323
Epoch 3/25
120/120 [==============================] - 9s 72ms/step - loss: 0.3529 - val_loss: 0.2561
Epoch 4/25
120/120 [==============================] - 9s 72ms/step - loss: 0.2763 - val_loss: 0.2019
Epoch 5/25
120/120 [==============================] - 9s 71ms/step - loss: 0.2237 - val_loss: 0.1689
Epoch 6/25
120/120 [==============================] - 9s 72ms/step - loss: 0.1850 - val_loss: 0.1342
Epoch 7/25
120/120 [==============================] - 9s 72ms/step - loss: 0.1551 - val_loss: 0.1182
Epoch 8/25
120/120 [==============================] - 9s 72ms/step - loss: 0.1313 - val_loss: 0.0922
Epoch 9/25
120/120 [==============================] - 9s 72ms/step - loss: 0.1171 - val_loss: 0.0846
Epoch 10/25
120/120 [==============================] - 9s 71ms/step - loss: 0.1023 - val_l

In [109]:
pred=model.predict(x_train)

In [110]:
for i in range(len(pred)):
    print(f"{i}, {pred[i]}, {y_train[i]}")



A streamkimeneten csak az utolsó 5000 sor látható.
5000, [9.9975473e-01 1.2762094e-04 1.1762319e-04], [1, 0, 0]
5001, [3.1257401e-05 9.7491953e-04 9.9899381e-01], [0, 0, 1]
5002, [0.0010595  0.00749451 0.991446  ], [0, 0, 1]
5003, [9.9980634e-01 7.0257061e-05 1.2351117e-04], [1, 0, 0]
5004, [9.9988663e-01 4.0750143e-05 7.2624411e-05], [1, 0, 0]
5005, [9.9720097e-01 2.1775158e-03 6.2150555e-04], [1, 0, 0]
5006, [9.9974436e-01 1.1175248e-04 1.4389686e-04], [1, 0, 0]
5007, [4.1820187e-04 6.3712792e-03 9.9321049e-01], [0, 0, 1]
5008, [9.9964201e-01 5.4186356e-05 3.0369728e-04], [1, 0, 0]
5009, [9.9940836e-01 1.5347637e-04 4.3814519e-04], [1, 0, 0]
5010, [9.9831867e-01 1.2719333e-03 4.0934552e-04], [1, 0, 0]
5011, [9.997793e-01 8.656544e-05 1.342484e-04], [1, 0, 0]
5012, [9.9925274e-01 1.9662056e-04 5.5054843e-04], [1, 0, 0]
5013, [9.998517e-01 7.527788e-05 7.289513e-05], [1, 0, 0]
5014, [9.99869943e-01 2.91023116e-05 1.00946156e-04], [1, 0, 0]
5015, [9.99841094e-01 5.37404158e-05 1.0512194

### Model usage

In [111]:
test_df=pd.read_csv("TEST.csv")

In [112]:
print(len(test_df))

2640


In [113]:
test_df["clean_text"]=sentences_list[:2640]

In [118]:
def code_sentences(df):
    all_sent=[]
    for sentence in df.clean_text:
        sent_list=sentence.strip().split()
        out=[]
        for word in sent_list:
            out.append(word_dict[word]) 
        all_sent.append(out)
    df["sentence_code"]=all_sent
    return (df)

In [119]:
test_df=code_sentences(test_df)

In [126]:
def push_sentences(df,lens=33):
    table=list(df["sentence_code"]) # typo
    
    print(lens)
    x0=[0 for _ in range(lens) ]
    x=[]
    for sent in table:
        o1=list(x0[0:lens-len(sent)]+list(sent))
        x.append(o1)
    return(x)

In [127]:
xx=push_sentences(test_df)

33
